In [1]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 97.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 73.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.0/94.0 kB 8.6 MB/s eta 0:00:

### Data Loading

In [2]:
from langchain_community.document_loaders import TextLoader
loader=TextLoader('Squirrel_text.txt')
document=loader.load()
print(document)

[Document(metadata={'source': 'Squirrel_text.txt'}, page_content="CompanyName:The Squirrel; The Squirrel is a dynamic, forward-thinking service-based company that excels in delivering a wide array of cutting-edge digital solutions, crafted to meet the diverse needs of modern businesses. With a focus on harnessing technology to its fullest potential, The Squirrel stands out as a one-stop destination for high-quality services ranging from machine learning model development, web development, app development, video editing, social media management, to SEO. Their deep commitment to innovation and precision enables them to build tailored machine learning models that help businesses optimize decision-making processes, enhance customer experiences, and streamline operations. From predictive analytics to natural language processing and deep learning applications, The Squirrel's machine learning team is adept at turning complex data into actionable insights that can transform the trajectory of a

### Indexing, data transformation

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
texts=text_splitter.split_documents(document)
print(texts)

[Document(metadata={'source': 'Squirrel_text.txt'}, page_content='CompanyName:The Squirrel; The Squirrel is a dynamic, forward-thinking service-based company that excels in delivering a wide array of cutting-edge digital solutions, crafted to meet the diverse needs of modern businesses. With a focus on harnessing technology to its fullest potential, The Squirrel stands out as a one-stop destination for high-quality services ranging from machine learning model development, web development, app development, video editing, social media management, to SEO. Their'), Document(metadata={'source': 'Squirrel_text.txt'}, page_content="to SEO. Their deep commitment to innovation and precision enables them to build tailored machine learning models that help businesses optimize decision-making processes, enhance customer experiences, and streamline operations. From predictive analytics to natural language processing and deep learning applications, The Squirrel's machine learning team is adept at tu

### Vector Embedding based on the model

In [8]:
import json
# !pip install Faiss_cpu
config_data=json.load(open("config.json"))
GOOGLE_API_KEY=config_data['GOOGLE_API_KEY']

from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS

embedding_model=GoogleGenerativeAIEmbeddings(model='models/embedding-001', google_api_key=GOOGLE_API_KEY)
db=FAISS.from_documents(texts,embedding_model)

### LLM and model

In [10]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import GoogleGenerativeAI

system="You're a helpful AI assistant of the company Squirrel, you help to resolve user query based on the company details. Use the context provided to answer the user query {context}"
human="{text}"

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", human),
    ]
)

llm=GoogleGenerativeAI(model='models/gemini-1.5-flash',temperature=0.4,google_api_key=GOOGLE_API_KEY)

# retriever = db.as_retriever()
# retriver_context=retriever.invoke("What is Squirrel?")
# print(retriver_context)

# Retriever for fetching documents from the database
retriever = db.as_retriever()

# Function to format the retrieved documents into a single string
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Retrieve the relevant context from the database for the query "What is The Squirrel?"
retriever_context = retriever.get_relevant_documents("About Squirrel?")
formatted_context = format_docs(retriever_context)

chain = prompt | llm | StrOutputParser()
chain.invoke({"context":retriever_context,"text":"What are the services provided by The Squirrel?"})

'The Squirrel offers a wide range of digital solutions for modern businesses, including:\n\n* **Machine learning model development**\n* **Web development**\n* **App development**\n* **Video editing**\n* **Social media management**\n* **SEO**\n\nThey also specialize in:\n\n* **Tailored strategies for brand loyalty, community engagement, and increased visibility across platforms.**\n* **SEO techniques to improve website visibility, attract organic traffic, and boost revenue.**\n* **User-centric approach to app development, delivering seamless and robust mobile and desktop applications.** \n'